In [16]:
from lark import Lark, UnexpectedEOF

ebnf_grammar = """
    document: module ANY
    module: "MODULE" IDENT ";" (import_list)? decl_seq ("BEGIN" statement_seq)? ("CLOSE" statement_seq)? "END" IDENT "."
    import_list: "IMPORT" (IDENT ":=")? IDENT ("," (IDENT ":=")? IDENT)* ";"
    decl_seq: ("CONST" (const_decl ";")* | "TYPE" (type_decl ";")* | "VAR" (var_decl ";")*)* (proc_decl ";" | forward_decl ";")*
    const_decl: ident_def "=" const_expr
    type_decl: ident_def "=" type
    var_decl: ident_list ":" type
    proc_decl: "PROCEDURE" (receiver)? ident_def (formal_pars)? meth_attributes (";" decl_seq ("BEGIN" statement_seq)? "END" IDENT)?
    meth_attributes: ("," "NEW")? ("," ("ABSTRACT" | "EMPTY" | "EXTENSIBLE"))?
    forward_decl: "PROCEDURE" " ^ " (receiver)? ident_def (formal_pars)? meth_attributes
    formal_pars: "(" (f_p_section (";" f_p_section)*)? ")" (":" type)?
    f_p_section: ("VAR" | "IN" | "OUT")? IDENT ("," IDENT)* ":" type
    receiver: "(" ("VAR" | "IN")? IDENT ":" IDENT ")"
    type: qualident | "ARRAY" (const_expr ("," const_expr)*)? "OF" type | ("ABSTRACT" | "EXTENSIBLE" | "LIMITED")? | "RECORD" ("(" qualident ")")? field_list (";" field_list)* "END" | "POINTER" "TO" type | "PROCEDURE" (formal_pars)?
    field_list: (ident_list ":" type)?
    statement_seq: statement (";" statement)*
    statement: (designator ":=" expr | designator ("(" (expr_list)? ")")? 
            | "IF" expr "THEN" statement_seq ("ELSIF" expr "THEN" statement_seq)* ("ELSE" statement_seq)? "END" 
            | "CASE" expr "OF" case ("|" case)* ("ELSE" statement_seq)? "END" 
            | "WHILE" expr "DO" statement_seq "END" 
            | "REPEAT" statement_seq "UNTIL" expr 
            | "FOR" IDENT ":=" expr "TO" expr ("BY" const_expr)? "DO" statement_seq "END" 
            | "LOOP" statement_seq "END" 
            | "WITH" (guard "DO" statement_seq)? ("|" (guard "DO" statement_seq)?)* ("ELSE" statement_seq)? "END" 
            | "EXIT" 
            | "RETURN" (expr)?)?
    case: (case_labels ("," case_labels)* ":" statement_seq)?
    case_labels: const_expr (".." const_expr)?
    guard: qualident ":" qualident
    const_expr: expr
    expr: simple_expr (relation simple_expr)?
    simple_expr: ("+" | "-")? term (add_op term)*
    term: factor (mul_op factor)*
    factor: designator | NUMBER | CHARACTER | STRING | "NIL" | set | "(" expr ")" | " ~ " factor
    set: "{" (element ("," element)*)? "}"
    element: expr (".." expr)?
    relation: "=" | "#" | "<" | "<=" | ">" | ">=" | "IN" | "IS"
    add_op: "+" | "-" | "OR"
    mul_op: " * " | "/" | "DIV" | "MOD" | "&"
    designator: qualident ("." IDENT | "[" expr_list "]" | " ^ " | "(" qualident ")" | "(" (expr_list)? ")")* ("$")?
    expr_list: expr ("," expr)*
    ident_list: ident_def ("," ident_def)*
    qualident: (IDENT ".")? IDENT
    ident_def: IDENT (" * " | "-")?

    ANY: /[^ ]+/    
    IDENT: CNAME
    NUMBER: INT | FLOAT | HEX
    HEX: /0x[0-9A-Fa-f]+/
    CHARACTER: /'[^']*'/
    STRING: /"[^"]*"/
    
    %import common.CNAME
    %import common.INT
    %import common.FLOAT
    %import common.WS
    
    %ignore WS
"""
#COMMENT: "(*" /(.|\n)*?/ "*)"
#%ignore COMMENT
comment_grammar = """
    COMMENT : "(*" /(.|\n)*?/ "*)"
    %import common.WS
    %ignore WS
"""


In [17]:
#, parser='lalr'
# lexer='basic', 

starts = [
    "module", 
    "import_list",
    "expr",
    "statement"
]





In [18]:
def rule_match(grammar, rule, text):
    parser = Lark(grammar, start=rule)
    try:
        tree = parser.parse(code)
        return 1
    except UnexpectedEOF:
        return 2
    except:
        return 0

def is_document(text):
    return rule_match(ebnf_grammar, "document",text) == 1

def is_module(text):
    return rule_match(ebnf_grammar, "module",text) == 1

def is_statement(text):
    return rule_match(ebnf_grammar, "statement",text) == 1

def get_module_text_len(text):
    parser = Lark(ebnf_grammar, start="document", propagate_positions=True)
    tree = parser.parse(text)
    return len(text) - len(tree.children[0].value)
    
def is_comment(text):
    return rule_match(comment_grammar, "COMMENT",text) == 1

def is_comment_start(text):
    return rule_match(comment_grammar, "COMMENT",text) == 2



def try_module(text, rule):
    parser = Lark(ebnf_grammar, start=rule)
    try:
        tree = parser.parse(code)
        return 1
    except UnexpectedEOF:
        return 2
    except:
        return 0

def load_file(path):
   with open(path, 'r', encoding='utf-8') as f:
      return f.read()

In [19]:
# Пример использования
code = """
MODULE HelloWorld;
IMPORT Out;
BEGIN
  a := b;
END HelloWorld.
"""
rule_match(ebnf_grammar, "document", code)

1

In [20]:
parser = Lark(ebnf_grammar, start="document", propagate_positions=True)
tree = parser.parse(code)

In [21]:
text = load_file('/app/datasets/oberon/docs/bb_ru/Dev/Mod/AlienTool.odc')

FileNotFoundError: [Errno 2] No such file or directory: '/app/datasets/oberon/docs/bb_ru/Dev/Mod/AlienTool.odc'

In [2]:
is_document(text)

NameError: name 'text' is not defined

In [104]:
rule_match(text, "document")

2